<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-198ce3b09edde99a'
PROJECT = 'qwiklabs-gcp-198ce3b09edde99a'
REGION = 'europe-north1-a'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [6]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,5896567601480310696,342825
1,9068386407968572094,330863
2,2700219941840496344,307395
3,260598435387740869,320332
4,454960867574323744,346515


In [7]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13204 examples in the train dataset and 3421 in the eval dataset


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [8]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.311835,False,14,1,39.0,3095933535584005890
1,6.311835,False,14,1,39.0,7420272703711713305
2,7.187070,True,14,1,40.0,454960867574323744
3,7.255413,True,14,1,43.0,5937540421097454372
4,2.250920,True,14,2,28.0,8387817883864991792


In [9]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13194.000000,13204.000000,13204.000000,13104.000000,1.320400e+04
mean,7.222134,27.442745,1.036883,38.605311,4.426430e+18
std,1.338229,6.163356,0.206139,2.594570,2.805451e+18
min,0.573202,14.000000,1.000000,19.000000,1.244589e+17
25%,6.563162,23.000000,1.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,7.170970e+18
max,12.202586,50.000000,5.000000,47.000000,9.183606e+18


In [10]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [11]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.311835,False,14,Single(1),39.0,3095933535584005890
1,6.311835,False,14,Single(1),39.0,7420272703711713305
2,7.187070,True,14,Single(1),40.0,454960867574323744
3,7.255413,True,14,Single(1),43.0,5937540421097454372
4,2.250920,True,14,Twins(2),28.0,8387817883864991792


In [12]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13199,8.375361,Unknown,46,Single(1),37.0,6888635180361964765
13200,2.438313,Unknown,47,Multiple(2+),22.0,8029892925374153452
13201,4.186578,Unknown,47,Single(1),38.0,2599641534368510197
13202,6.395610,Unknown,48,Single(1),40.0,2768780346491825728
13203,7.374463,Unknown,50,Single(1),39.0,1077881854928885650


In [13]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26194.000000,26194.000000,26194.000000,2.619400e+04
mean,7.224076,27.450714,38.606169,4.425621e+18
std,1.334827,6.163122,2.591356,2.805220e+18
min,0.573202,14.000000,19.000000,1.244589e+17
25%,6.563162,23.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,40.000000,7.170970e+18
max,12.202586,50.000000,47.000000,9.183606e+18


In [14]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [15]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6780 eval.csv
  26194 train.csv
  32974 total
==> eval.csv <==
7.68751907594,False,14,Single(1),39.0,4740473290291881219
7.81318256528,True,16,Single(1),39.0,4740473290291881219
7.6941329438,True,17,Single(1),40.0,4740473290291881219
7.6500404913999995,True,17,Single(1),41.0,4740473290291881219
10.43668348308,False,17,Single(1),38.0,4740473290291881219
7.43839671988,True,17,Single(1),40.0,4740473290291881219
5.93704871566,True,18,Single(1),38.0,4740473290291881219
7.12534030784,True,18,Single(1),36.0,4740473290291881219
5.8753192823,False,19,Single(1),37.0,4740473290291881219
7.12534030784,True,19,Single(1),43.0,4740473290291881219

==> train.csv <==
6.3118345610599995,False,14,Single(1),39.0,3095933535584005890
6.3118345610599995,False,14,Single(1),39.0,7420272703711713305
7.1870697412,True,14,Single(1),40.0,454960867574323744
7.25541304242,True,14,Single(1),43.0,5937540421097454372
2.25091969502,True,14,Twins(2),28.0,8387817883864991792
7.49792153062,False,14,Single(1),28.0,357245

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License